In [ ]:
path = "D:\models\python"
import pickle
import keras
import tensorflow as tf
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
import os
import json
import sys
import time
import datetime
import shutil
import subprocess
import re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import math





In [ ]:
#load 1 model
modelName = "python_model_1"
model = load_model(path + modelName + '/model.h5')
model.summary()
import pickle
with open(path + modelName + '/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
with open(path + modelName + '/modelInfo.json') as json_file:
    modelInfo = json.load(json_file)

In [ ]:
"load Array of models"

In [ ]:
"Test Models"
